In [1]:
from models.modeling_dsm import DSM
from torchinfo import summary

In [2]:
model = DSM.from_pretrained("GleghornLab/DSM_650")
model

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

You are using a model of type esm_diff to instantiate a model of type dsm. This is not supported for all configurations of models and can yield errors.


model.safetensors:   0%|          | 0.00/2.61G [00:00<?, ?B/s]

DSM(
  (esm): FAST_ESM_ENCODER(
    (embeddings): EsmEmbeddings(
      (word_embeddings): Embedding(33, 1280, padding_idx=1)
    )
    (encoder): EsmEncoder(
      (layer): ModuleList(
        (0-32): 33 x EsmLayer(
          (attention): EsmAttention(
            (self): EsmSelfAttention(
              (query): Linear(in_features=1280, out_features=1280, bias=True)
              (key): Linear(in_features=1280, out_features=1280, bias=True)
              (value): Linear(in_features=1280, out_features=1280, bias=True)
              (rotary_embeddings): RotaryEmbedding()
            )
            (output): EsmSelfOutput(
              (dense): Linear(in_features=1280, out_features=1280, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (LayerNorm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          )
          (intermediate): EsmIntermediate(
            (dense): Linear(in_features=1280, out_features=5120, bias=True)
          )
   

In [3]:
summary(model)

Layer (type:depth-idx)                                       Param #
DSM                                                          --
├─FAST_ESM_ENCODER: 1-1                                      --
│    └─EsmEmbeddings: 2-1                                    --
│    │    └─Embedding: 3-1                                   42,240
│    └─EsmEncoder: 2-2                                       --
│    │    └─ModuleList: 3-2                                  649,355,520
│    │    └─LayerNorm: 3-3                                   2,560
│    └─EsmContactPredictionHead: 2-3                         --
│    │    └─Linear: 3-4                                      661
│    │    └─Sigmoid: 3-5                                     --
├─EsmPooler: 1-2                                             --
│    └─Linear: 2-4                                           1,639,680
│    └─Tanh: 2-5                                             --
├─LMHead: 1-3                                                --
│    └─Line

In [1]:
# Visualize a protein structure CIF file and highlight annotated domains/regions

import nglview as nv

# Path to your CIF file
cif_file = "model.cif"  # <-- change this to your file path

# Load the structure
view = nv.show_file(cif_file)

# Define regions and their colors
# Format: (start_residue, end_residue, color, label)
regions = [
    (2, 223, "yellow", "Region: 2-223 (SART31 interaction)"),
    (7, 118, "orange", "Domain: 7-118 (DUSP PROSITE)"),
    (216, 237, "magenta", "Region: 216-237 (Disordered)"),
    (226, 237, "cyan", "Compositional bias: 226-237 (Polar)"),
    (289, 933, "green", "Domain: 289-933 (USP PROSITE)"),
    (629, 694, "magenta", "Region: 629-694 (Disordered)"),
    (656, 673, "red", "Compositional bias: 656-673 (Acidic)"),
    (952, 981, "magenta", "Region: 952-981 (Disordered)"),
    (960, 974, "red", "Compositional bias: 960-974 (Acidic)"),
]

# Add cartoon representation for the whole structure
view.clear_representations()
view.add_cartoon(color="lightgrey", opacity=0.3)

# Highlight each region
for start, end, color, label in regions:
    # NGL residue numbers are usually 1-based and inclusive
    selection = f"{start}-{end}"
    view.add_cartoon(selection=selection, color=color)
    # Optionally, add a label at the center residue
    center = (start + end) // 2
    view.add_label(selection=f"{center}", color=color, labelType="residue", labelText=label)

# Show the viewer
view


NGLWidget()

In [2]:
from datasets import load_dataset

In [ ]:
data = load_dataset("Synthyra/BIOGRID", split="train")
data


In [ ]:
seqs = set()

data = data.shuffle(seed=42)

def process_dataset(ex):
    seq_a, seq_b = ex['SeqA'], ex['SeqB']
    if seq_a in seqs or seq_b in seqs:
        return False
    seqs.add(seq_a)
    seqs.add(seq_b)
    return True

data = data.filter(process_dataset)
data

In [ ]:
data = load_dataset('Synthyra/all_string_pairs_700', split='train')
data

In [ ]:
class UniqueIDFilter:
    def __init__(self):
        self.ids_seen = set()
    
    def __call__(self, ex):
        if ex['score'] < 900:
            return False
        id_a, id_b = ex['pairs'].split('|')
        if id_a in self.ids_seen or id_b in self.ids_seen:
            return False
        self.ids_seen.add(id_a)
        self.ids_seen.add(id_b)
        return True


filter_func = UniqueIDFilter()
data = data.filter(filter_func, num_proc=16)
data

In [ ]:
data = load_dataset('Synthyra/all_string_pairs_900_unique', split='train')
ids = set()
for pair in data['pairs']:
    id_a, id_b = pair.split('|')
    ids.add(id_a)
    ids.add(id_b)
del data
len(ids)

In [ ]:
string_seqs = load_dataset('Synthyra/StringDBSeqsv12', split='train')


def get_relevant_ids(ex):
    id = ex['id']
    if id in ids:
        return True
    return False

string_seqs = string_seqs.filter(get_relevant_ids)
string_seqs

In [ ]:
string_seqs.push_to_hub('Synthyra/StringDBSeqsv12_unique', private=True) # 20% ish in av set

In [ ]:
seqs = set(load_dataset('Synthyra/StringDBSeqsv12_unique', split='train')['sequence']) # 3% ish in av set
len(seqs)

In [3]:
seqs = set(load_dataset('lhallee/foldseek_dataset', split='train')['seqs']) # 3% ish in av set
len(seqs)

README.md:   0%|          | 0.00/519 [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

train-00000-of-00017.parquet:   0%|          | 0.00/413M [00:00<?, ?B/s]

train-00001-of-00017.parquet:   0%|          | 0.00/413M [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/17070828 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/474 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/474 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/17 [00:00<?, ?it/s]

17069128

In [ ]:
from tqdm.auto import tqdm


av = load_dataset('lhallee/AV_large', split='train', streaming=True)

counter, total = 0, 0

pbar = tqdm(av)
for ex in pbar:
    if ex['sequence'] in seqs:
        counter += 1
    total += 1
    pbar.set_description(f'{counter}/{total}')

counter


Resolving data files:   0%|          | 0/42 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/42 [00:00<?, ?it/s]

0it [00:00, ?it/s]